In [23]:
#!/usr/bin/env python
# encoding: utf-8
from __future__ import absolute_import, division, print_function, unicode_literals
# Install TensorFlow
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator



from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
# from xgboost import XGBClassifier
import tensorflow.keras.backend as K
from sklearn import metrics

# !pip3 install keras-tuner --upgrade
# !pip3 install autokeras
import kerastuner as kt
import autokeras as ak

# Import local libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import importlib
from tqdm import tqdm 
import os
import sys
import logging

importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

os.environ['NUMEXPR_MAX_THREADS'] = '64'
os.environ['NUMEXPR_NUM_THREADS'] = '64'


gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)



logging.info("Tensorflow Version is {}".format(tf.__version__))
logging.info("Keras Version is {}".format(tf.keras.__version__))
from tensorflow.python.client import device_lib
logging.info(device_lib.list_local_devices())
tf.device('/device:XLA_GPU:0')

!nvidia-smi

INFO:root:Tensorflow Version is 2.4.1
INFO:root:Keras Version is 2.4.0
INFO:root:[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7824930124829175963
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10485760000
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 49387416044665903
physical_device_desc: "device: 0, name: A100-SXM-80GB, pci bus id: 0000:48:00.0, compute capability: 8.0"
]


1 Physical GPUs, 1 Logical GPU
Sun Dec 12 08:27:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM-80GB       On   | 00000000:48:00.0 Off |                    0 |
| N/A   34C    P0    94W / 400W |  43061MiB / 81251MiB |     97%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                         

Ref: https://keras.io/keras_tuner/#quick-introduction  
Ref: https://keras.io/guides/keras_tuner/getting_started/   
Ref: https://keras.io/api/keras_tuner/tuners/base_tuner/

In [5]:
def Loading_Data(data_source, datadict, start=0, stop=20000):
    x_jet, target = [], []

    time.sleep(0.5)
    for k in tqdm(range(start,len(data_source))):
        x_jet_path = savepath + "Image_Directory/"+ data_source["JetImage"].iloc[k]
        x_jet_tmp = np.load(x_jet_path)["jet_image"]
        if np.isnan(x_jet_tmp).any() == True:
            continue 

        target.append(data_source["Y"].iloc[k])
        x_jet_tmp = np.divide((x_jet_tmp - Norm_dict[datadict][0]), (np.sqrt(Norm_dict[datadict][1])+1e-5))#[0].reshape(1,40,40)
        x_jet.append(x_jet_tmp)


        if k == stop:
            break

    return np.asarray(x_jet), np.asarray(target)



In [4]:
%%time
HOMEPATH = "/dicos_ui_home/alanchung/Universality_Boosetd_Higgs/"
JetImagePath =  HOMEPATH + "Data_ML/" +"Image_Directory/"
savepath = HOMEPATH + "Data_ML/"


    
try:
    
    data_dict ={
#             "herwig_ang" : [0,0],
            "pythia_def" : [0,0],
#             "pythia_vin" : [0,0],
#             "pythia_dip" : [0,0],
#             "sherpa_def" : [0,0],
              }  
    
    Norm_dict ={
#             "herwig_ang" : [0,0],
            "pythia_def" : [0,0],
#             "pythia_vin" : [0,0],
#             "pythia_dip" : [0,0],
#             "sherpa_def" : [0,0],
              }  
    
    data_train = {
#             "herwig_ang_train" : 0,
            "pythia_def_train" : 0,
#             "pythia_vin_train" : 0,
#             "pythia_dip_train" : 0,
#             "sherpa_def_train" : 0
            }  
    

    for i, element in enumerate(data_dict):
        data_dict[element][0] = pd.read_csv(savepath + str(element) + "_H_dict.csv")
        data_dict[element][1] = pd.read_csv(savepath + str(element) + "_QCD_dict.csv")
#         logging.info(len(data_dict[element][0]),len(data_dict[element][1]))
    
    for i, element in enumerate(Norm_dict):
        average_H = np.load(savepath + "average" + "_" + str(element) + "_H.npy")
        variance_H = np.load(savepath + "variance" + "_" + str(element) + "_H.npy")
        average_QCD = np.load(savepath + "average" + "_" + str(element) + "_QCD.npy")
        variance_QCD = np.load(savepath + "variance" + "_" + str(element) + "_QCD.npy")
        length_H = len(data_dict[element][0])
        length_QCD = len(data_dict[element][1])
        
        Norm_dict[element][0] = (average_H*length_H + average_QCD*length_QCD)/(length_H+length_QCD)
        Norm_dict[element][1] =  variance_H + variance_QCD
        
    for i,(element, dict_element) in enumerate(zip(data_train, data_dict)):
        
        """
        Pt Range Study
        """
        pt_min, pt_max = 300, 500
        tmp = pd.read_csv(HOMEPATH + "Notebook/KFold_CNN/" + str(element) + ".csv")
        tmp = tmp[(tmp["PTJ_0"] >= pt_min)  & (tmp["PTJ_0"] < pt_max)]
        tmp = tmp[(tmp["MJ_0"] >= 110)  & (tmp["MJ_0"] < 160)]
        data_train[element] = shuffle(tmp)
        
        H_tmp = data_train[element][data_train[element]["target"] == 1]
        QCD_tmp = data_train[element][data_train[element]["target"] == 0]
        
        H_dict = data_dict[dict_element][0].iloc[H_tmp["index"].values]
        QCD_dict = data_dict[dict_element][1].iloc[QCD_tmp["index"].values]
        
        data_train[element] = pd.concat([H_dict, QCD_dict], ignore_index=True, axis=0,join='inner')
        data_train[element] = shuffle(data_train[element])
        


    logging.info("All Files are loaded!")

    logging.info("H jet : QCD jet = 1 : 1")
    logging.info("\r")
    train = [ len(data_train[element]) for j, element in enumerate(data_train)]
    logging.info("{:^8}{:^15}".format("",str(element)))
    logging.info("{:^8}{:^15}".format("Train #",train[0]))
    
    
    for i, element in enumerate(data_train):
        total_list = data_train[element].columns
        break
    
    logging.info("total_list {}".format(total_list))

except:
    
    logging.info("Please create training, test and validation datasets.")


INFO:root:All Files are loaded!
INFO:root:H jet : QCD jet = 1 : 1
INFO:root:
INFO:root:        pythia_def_train
INFO:root:Train #     307456     
INFO:root:total_list Index(['JetImage', 'Y'], dtype='object')


CPU times: user 2.77 s, sys: 309 ms, total: 3.08 s
Wall time: 3.08 s


In [54]:
%%time

# x_jet, target = Loading_Data(data_train["pythia_def_train"], "pythia_def", start=0, stop= len(data_train["pythia_def_train"]))
x_jet, target = Loading_Data(data_train["pythia_def_train"], "pythia_def", start=0, stop= 50000)



 16%|█▋        | 50000/307456 [05:33<28:38, 149.83it/s] 


CPU times: user 32.9 s, sys: 6.53 s, total: 39.4 s
Wall time: 5min 35s


In [55]:
%%time


length = len(x_jet)
x_train_jet, target_train = x_jet[:int(length/10*8)], target[:int(length/10*8)]
x_val_jet, target_val = x_jet[int(length/10*8):int(length/10*9)], target[int(length/10*8):int(length/10*9)]
x_test_jet, target_test = x_jet[int(length/10*9):], target[int(length/10*9):]

logging.info("training length: {}".format(len(x_train_jet)))
logging.info("validation length: {}".format(len(x_val_jet)))
logging.info("test length: {}".format(len(x_test_jet)))
logging.info("Total length: {}".format(len(x_train_jet)+len(x_val_jet)+len(x_test_jet)))
logging.info("Total length: {}".format(length))



INFO:root:training length: 40000
INFO:root:validation length: 5000
INFO:root:test length: 5001
INFO:root:Total length: 50001
INFO:root:Total length: 50001


CPU times: user 1.73 ms, sys: 2.63 ms, total: 4.36 ms
Wall time: 3.31 ms


In [92]:
"""
Define Function
"""

def CNN_Model(hp):
    """
    Declare the Input Shape
    """
    input_shape = (3,40,40)#(1, 40,40)


    """
    Create a Sequential Model
    """
    model_CNN = Sequential(name = "Model_CNN_Pythia_Default")


    """
    Add a "Conv2D" Layer into the Sequential Model
    """
    model_CNN.add(Conv2D(
                     filters=hp.Int("Conv2D_1_filter", min_value=16, max_value=128, step=16), 
                     kernel_size=hp.Choice('Conv2D_1_kernel', values = [3,5,7]),
                     strides=hp.Choice('Conv2D_1_stride', values = [1]),
                     activation='relu',
                     data_format='channels_first',
    #                data_format='channels_last',
                    input_shape=input_shape, 
                    name = 'Conv2D_1'))

    """
    Add a "MaxPooling2D" Layer into the Sequential Model
    """
    model_CNN.add(MaxPooling2D(pool_size=(2, 2), 
                               strides=(2, 2),
#                                data_format='channels_first', 
    #                            data_format='channels_last',
                               name = 'jet_MaxPooling_1'))

    """
    Add a "Conv2D" Layer into the Sequential Model
    """
    model_CNN.add(Conv2D(
                 filters=hp.Int("Conv2D_2_filter", min_value=16, max_value=128, step=16), 
                 kernel_size=hp.Choice('Conv2D_2_kernel', values = [3,5,7]),
                 strides=hp.Choice('Conv2D_2_stride', values = [1]),
                 activation='relu',
                 data_format='channels_first',
#                data_format='channels_last',
                input_shape=input_shape, 
                name = 'Conv2D_2'))

    """
    Add a "MaxPooling2D" Layer into the Sequential Model
    """
    model_CNN.add(MaxPooling2D(pool_size=(2, 2), 
                               strides=(2, 2),
                               data_format='channels_first', 
    #                            data_format='channels_last',
                               name = 'jet_MaxPooling_2'))



    """
    Flatten
    """
    model_CNN.add(Flatten(name = 'jet_flatten'))


    """
    Put Output from Flatten Layer into "Dense" Layer with 300 neurons
    """
    for i in range(hp.Int("num_layers", 1, 3)):
        model_CNN.add(
            keras.layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=100, max_value=500, step=50),
                activation='relu',
            )
        )

    
    if hp.Boolean("dropout"):
        model_CNN.add(keras.layers.Dropout(rate=0.01, name = 'Dropout'))

    """
    Add Output "Dense" Layer with 2 neurons into the Sequential Model
    """
    model_CNN.add(Dense(1, activation='sigmoid', name = 'output'))
    # model_CNN.add(Dense(2, activation='softmax', name = 'output'))

    """
    Declare the Optimizer
    """
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
    model_opt = keras.optimizers.Adadelta(learning_rate=learning_rate)
    # model_opt = keras.optimizers.Adam()


    """
    Compile Model
    """
    model_CNN.compile(
    #                     loss="categorical_crossentropy",
                       loss = "binary_crossentropy",
                       optimizer=model_opt,
                       metrics=["accuracy","mse"])

    """
    Print Architecture
    """
    model_CNN.summary()

    return model_CNN

In [93]:
%%time
tuner = kt.RandomSearch(hypermodel=CNN_Model,
                        objective="val_loss",
                        max_trials=3,
                        executions_per_trial=2, #The number of models that should be built and fit for each trial
                        overwrite=True,
                        directory="CNN_Model_Hyper_Tunning",
                        project_name="Universality"
                        )
tuner.search_space_summary()

Model: "Model_CNN_Pythia_Default"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_1 (Conv2D)            (None, 16, 38, 38)        448       
_________________________________________________________________
jet_MaxPooling_1 (MaxPooling (None, 8, 19, 38)         0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 17, 36)        1168      
_________________________________________________________________
jet_MaxPooling_2 (MaxPooling (None, 16, 8, 18)         0         
_________________________________________________________________
jet_flatten (Flatten)        (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               230500    
_________________________________________________________________
output (Dense)               (None, 1)    

In [94]:
tuner.search(np.asarray(x_train_jet), np.asarray(target_train),
             epochs=5, 
             validation_data=(np.asarray(x_val_jet), np.asarray(target_val)))

Trial 3 Complete [00h 00m 45s]
val_loss: 0.6931541264057159

Best val_loss So Far: 0.6799198985099792
Total elapsed time: 00h 02m 20s
INFO:tensorflow:Oracle triggered exit


In [95]:
tuner.results_summary()

Results summary
Results in CNN_Model_Hyper_Tunning/Universality
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
Conv2D_1_filter: 48
Conv2D_1_kernel: 7
Conv2D_1_stride: 1
Conv2D_2_filter: 96
Conv2D_2_kernel: 7
Conv2D_2_stride: 1
num_layers: 1
units_0: 400
dropout: False
lr: 0.001576691183648427
units_1: 250
Score: 0.6799198985099792
Trial summary
Hyperparameters:
Conv2D_1_filter: 128
Conv2D_1_kernel: 7
Conv2D_1_stride: 1
Conv2D_2_filter: 32
Conv2D_2_kernel: 3
Conv2D_2_stride: 1
num_layers: 2
units_0: 350
dropout: True
lr: 0.0015407870950216247
units_1: 100
Score: 0.68257936835289
Trial summary
Hyperparameters:
Conv2D_1_filter: 64
Conv2D_1_kernel: 7
Conv2D_1_stride: 1
Conv2D_2_filter: 32
Conv2D_2_kernel: 5
Conv2D_2_stride: 1
num_layers: 1
units_0: 200
dropout: True
lr: 2.3386744927248327e-05
units_1: 100
Score: 0.6931541264057159


In [96]:
best_model = tuner.get_best_models(num_models=3)[0]
# len(best_model)
# best_model[0].summary()

Model: "Model_CNN_Pythia_Default"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_1 (Conv2D)            (None, 48, 34, 34)        7104      
_________________________________________________________________
jet_MaxPooling_1 (MaxPooling (None, 24, 17, 34)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 96, 11, 28)        112992    
_________________________________________________________________
jet_MaxPooling_2 (MaxPooling (None, 96, 5, 14)         0         
_________________________________________________________________
jet_flatten (Flatten)        (None, 6720)              0         
_________________________________________________________________
dense (Dense)                (None, 400)               2688400   
_________________________________________________________________
output (Dense)               (None, 1)    

In [97]:
prediction_test =  best_model.predict(np.asarray(x_test_jet))
discriminator_test = prediction_test
discriminator_test = discriminator_test/(max(discriminator_test))


In [98]:
Performance_Frame = {
                "AUC" : [0],
                "max_sig" : [0],
                "r05" : [0],
                }
Performance_Frame["AUC"][0] = metrics.roc_auc_score(target_test,discriminator_test)
FalsePositiveFull, TruePositiveFull, _ = metrics.roc_curve(target_test,discriminator_test)
tmp = np.where(FalsePositiveFull != 0)
Performance_Frame["max_sig"][0] = max(TruePositiveFull[tmp]/np.sqrt(FalsePositiveFull[tmp])) 
tmp = np.where(TruePositiveFull >= 0.5)
Performance_Frame["r05"][0]= 1./FalsePositiveFull[tmp[0][0]]


dataframe = pd.DataFrame(Performance_Frame)
dataframe

,AUC,max_sig,r05
0,0.815317,1.567296,8.88968


In [99]:
best_model.save("./Tuning.h5")
model = load_model("./Tuning.h5", compile =False)
model.summary()

Model: "Model_CNN_Pythia_Default"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_1 (Conv2D)            (None, 48, 34, 34)        7104      
_________________________________________________________________
jet_MaxPooling_1 (MaxPooling (None, 24, 17, 34)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 96, 11, 28)        112992    
_________________________________________________________________
jet_MaxPooling_2 (MaxPooling (None, 96, 5, 14)         0         
_________________________________________________________________
jet_flatten (Flatten)        (None, 6720)              0         
_________________________________________________________________
dense (Dense)                (None, 400)               2688400   
_________________________________________________________________
output (Dense)               (None, 1)    

In [100]:
prediction_test =  model.predict(np.asarray(x_test_jet))
discriminator_test = prediction_test
discriminator_test = discriminator_test/(max(discriminator_test))
Performance_Frame = {
                "AUC" : [0],
                "max_sig" : [0],
                "r05" : [0],
                }
Performance_Frame["AUC"][0] = metrics.roc_auc_score(target_test,discriminator_test)
FalsePositiveFull, TruePositiveFull, _ = metrics.roc_curve(target_test,discriminator_test)
tmp = np.where(FalsePositiveFull != 0)
Performance_Frame["max_sig"][0] = max(TruePositiveFull[tmp]/np.sqrt(FalsePositiveFull[tmp])) 
tmp = np.where(TruePositiveFull >= 0.5)
Performance_Frame["r05"][0]= 1./FalsePositiveFull[tmp[0][0]]


dataframe = pd.DataFrame(Performance_Frame)
dataframe

,AUC,max_sig,r05
0,0.815317,1.567296,8.88968
